In [2]:
import yaml

In [12]:
# Path to your YAML configuration file
config_file_path = r'D:\kasia\tracking\E6_exp\sample_data\config.yaml'

In [20]:
# Reading the YAML file
with open(config_file_path, 'r') as file:
    config_data = yaml.safe_load(file)

# Accessing data from the configuration
experiment_name = config_data['experiment_settings']['experiment_name']
signal_channels = config_data['signal_channels']
graphs = config_data['graphs']

In [21]:
graphs

[{'name': 'Area', 'db_features': "-'Area'"},
 {'name': 'DHB', 'db_features': ['signal1_nuc', 'signal1_cyto']}]